# Torch.NN
- keras같이 유연한 모델 설계 가능하게 Pytorch에서 지원

In [1]:
from pathlib import Path
import requests

DATA_PATH = Path('data')
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

In [2]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [3]:
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(x_train[0].reshape((28,28)), cmap='gray')
print(x_train.shape)

(50000, 784)


In [4]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)


## Pytorch Nerual Network without torch
- using Pytorch Tensor

In [5]:
import math 

# requires_grad = True를 줌으로써 자동으로 gradient를 계산할 수 있게 함
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad = True)

- gardient를 자동으로 계산하는 Pytorch의 requires_grad 덕분에
    - 모든 표준 Python함수를 모델로 사용할 수 있음
    - 일반 행렬 곱셈(내적) 과 브로드 캐스트 덧셈 연산
    - cost function이나 activate function도 Python 통해 쉽게 작성 가능

In [6]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    # @은 내적연산
    return log_softmax(xb @ weights + bias)

In [7]:
bs = 64 # batch size

xb = x_train[0:bs] # mini-batch
preds = model(xb)
print(preds[0], preds.shape)

tensor([-2.2226, -1.7241, -2.7330, -2.7850, -2.8070, -1.8320, -2.1479, -2.6124,
        -2.3744, -2.4898], grad_fn=<SelectBackward>) torch.Size([64, 10])


- `preds` 텐서는 텐서 값뿐만 아니라 gradient function도 포함
    - grad_fn 통해 backpropagtion

In [8]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll

In [9]:
yb = y_train[0:bs]
print(loss_func(preds, yb))

tensor(2.3680, grad_fn=<NegBackward>)


In [10]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [11]:
print(accuracy(preds, yb))

tensor(0.0938)


### Training
1. batch size 설정 - mini batch
2. model을 사용하여 predict
3. 손실을 계산
4. 모델의 기울기 업데이트

In [12]:
from IPython.core.debugger import set_trace

lr = 0.5
epochs = 2

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        #set_trace()
        # ipython debugger : http://pythonstudy.xyz/python/article/505-Python-%EB%94%94%EB%B2%84%EA%B9%85-PDB
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        # backpropagation
        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

In [13]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0822, grad_fn=<NegBackward>) tensor(1.)


## torch.nn.functional 사용
- torch.nn.functional 통해 활성화 및 손실 함수를 불러옴
    - 이 모듈에는 torch.nn라이브러리의 모든 기능이 포함됨

In [14]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [15]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0822, grad_fn=<NllLossBackward>) tensor(1.)


## nn.Module을 사용한 리팩토링

In [16]:
from torch import nn


class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))
        
    def forward(self, xb):
        return xb @ self.weights + self.bias

In [17]:
model = Mnist_Logistic()

In [18]:
model(xb).shape

torch.Size([16, 10])

In [19]:
print(loss_func(model(xb), yb))

tensor(2.4019, grad_fn=<NllLossBackward>)


In [20]:
model = Mnist_Logistic()
# with torch.no_grad():
#     weights -= weights.grad * lr
#     bias -= bias.grad * lr
#     weights.grad_zero_()
#     bias.grad_zero_()
with torch.no_grad():
    for p in model.parameters():
        p -= p * lr
    model.zero_grad()

In [21]:
def fit():
    for epoch in range(epochs):
        for i in range((n -1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)
            
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()
fit()

In [22]:
print(loss_func(model(xb), yb))

tensor(0.0807, grad_fn=<NllLossBackward>)


## nn.Linear를 사용하여 리팩터링
- 수동으로 정의하고 초기화 self.weight, self.bias 게산하는 대신 `nn.Linear` 레이어 사용하여 모든 것을 수행

In [23]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)
        
    def forward(self, xb):
        return self.lin(xb)

In [24]:
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

tensor(2.2450, grad_fn=<NllLossBackward>)


In [25]:
fit()

print(loss_func(model(xb), yb))

tensor(0.0807, grad_fn=<NllLossBackward>)


## 최적화를 사용하여 리팩터링
- `torch.optim`: 최적화 알고리즘 포함된 패키지
- `.step`: 각 매개변수를 자동으로 업데이트

In [26]:
from torch import optim

def get_model():
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

tensor(2.2986, grad_fn=<NllLossBackward>)


In [27]:
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
print(loss_func(model(xb), yb))

tensor(0.0808, grad_fn=<NllLossBackward>)


## 데이터 세트를 사용한 리팩토링
- Dataset에 대한 인덱싱, 슬라이싱을 손쉽게 할 수 있음

In [28]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(x_train, y_train)

In [29]:
# xb = x_train[start_i:end_i]
# yb = y_train[start_i:end_i]

xb, yb = train_ds[1*bs : 1*bs+bs]

In [30]:
print(xb.shape, yb.shape)

torch.Size([64, 784]) torch.Size([64])


In [31]:
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n-1) // bs+1):
#         xb = x_train[start_i:end_i]
#         yb = y_train[start_i:end_i]
        xb, yb = train_ds[i*bs :  i*bs+bs]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
#          with torch.no_grad():
#              for p in model.parameters(): p -= p.grad * lr
#              model.zero_grad()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0826, grad_fn=<NllLossBackward>)


## DataLoader를 사용하여 리팩터링
- 배치 관리를 담당

In [32]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

In [33]:
print(train_ds.__len__())
print(train_dl.__len__())

50000
782


In [34]:
# for i in range((n-1)//bs + 1):
#     xb,yb = train_ds[i*bs : i*bs+bs]
#     pred = model(xb)
for xb,yb in train_dl:
    pred = model(xb)

In [35]:
model, opt = get_model()

for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
print(loss_func(model(xb), yb))

tensor(0.0805, grad_fn=<NllLossBackward>)


## 유효성 검사 추가
- 셔플링이나, 역전파 필요하지않음
    - 효율적인 자원활용필요

In [36]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

In [37]:
model, opt = get_model()

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

0 tensor(0.2887)
1 tensor(0.3298)


## fit() 및 get_data() 만들기
- 훈련 세트와 검증 세트 모두에 대해 손실을 계산하는 두 `loss_batch`번의 유사한 프로세스를 거치므로, 이를 하나의 배치에 대한 손실을 계산
- 훈련 세트에 대한 optimizer 전달하고 이를 사용하여 backpropagation 수행

In [38]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    
    return loss.item(), len(xb)

- `fit` 모델을 교육하고 각 시대에 대한 교육 및 검증 손실을 계산하는데 필요한 작업을 실행

- `**` : dict 형태의 초기화
    - 함수선언 시 인자로 들어갈때는 선언되지 않은 인자를 dict형태로 받아내어 사용
    - 함수호출 시 dict형태로 인자 넣어주면 각 함수 선언된 매개변수에 값 할당
- `*` : tuple 형태의 초기화
    - `**`와 마찬가진데 dictinoary -> tuple로 바뀐다고 보면 됨
- `zip` : tuple형태의 값을 각 변수로 할당

In [39]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)
        
        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
    
        print(epoch, val_loss)

- `get_data` 교육 및 유효성 검사 세트에 대한 데이터 로더를 반환

In [40]:
def get_data(train_ds, valid_ds, bs):
    return(
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

-  데이터 로더를 얻고, 모델을 피팅하는 전체 프로세스

In [41]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.33376503516435624
1 0.31128500435352324


## CNN으로 전환

In [42]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)
        
    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

- momentum은 이전 업데이트 고려해 더 빠른 훈련

In [43]:
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.37163340134620665
1 0.2495137122154236


## nn.Sequential
- 순차적으로 각 모듈을 실행해 신경망을 간단히 만들 수 있음
    - 하지만, 사용자 정의레이어를 쉽게 정의할 수 있어야함
        - Pytorch에는 View layer가 없으니 클래스 만들어서 쓰던가 해야함

In [44]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
        
    def forward(self, x):
        return self.func(x)
    
def preprocess(x):
    return x.view(-1, 1, 28, 28)

In [45]:
model = nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.35011338419914245
1 0.2742500264286995


## 랩핑 DataLoader


In [61]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func
        
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))
            
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
print(type(train_dl), type(valid_dl))
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

<class 'torch.utils.data.dataloader.DataLoader'> <class 'torch.utils.data.dataloader.DataLoader'>


In [62]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [63]:
fit(epoch, model, loss_func, opt, train_dl, valid_dl)

0 0.34364815847873686


## GPU 사용

In [64]:
# gpu check
print(torch.cuda.is_available())

True


In [66]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [67]:
# preprocess 배치를 GPU로 이동하도록 업데이트
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(device), y.to(device)

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

In [68]:
# 모델을 GPU로 옮김
model.to(device)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [69]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.260062384223938
1 0.23672253460884093


## 요약

### torch.nn
- `Module` : 함수처럼 동작하지만 상태(레이어 가중치)를 포함할 수 있는 callable
    - model의 parameter를 알 수 있고 
    - 모든 gradient를 0으로 만들고 weight 업데이트를 위해 gradient 반복
- `Parameter` : `Module` backpropagation 중 업데이트해야하는 weight가 있음을 알려주는 wrapper for a tensor
    - requires_grad속성이 설정된 텐서만 업데이트
- `functional` : `F`활성화 함수, 손실 함수 등 뿐만 아니라 컨벌루션 및 선형 레이어와 같은 비 상태 버전의 레이어를 포함하는 모듈(일반적으로 규칙에 따라 네임스페이스로 가져옴)

### torch.optim
- `SGD`같은 최적화가 들어있어 `Parameter`의 backpropagtion단계에서 가중치를 갱신

### Dataset
- Pytorch와 함께 제공되는 클래스를 포함하며 Dataset을 나타내는 추상클래스
    - 객체의 추상 인터페이스 `__len__`, `__getitem__`

### DataLoader
- `Dataset`데이터를 가져와 일괄처리를 반복하는 반복자를 만듬